### Understanding the data

In [42]:
import json
from IPython.display import JSON
import pprint

In [2]:
with open('flickr8k/dataset.json', 'r') as f:
    json_data = json.load(f)

In [4]:
json_data.keys()

dict_keys(['images', 'dataset'])

In [12]:
json_data['dataset']

'flickr8k'

In [38]:
json_data['images'][0].keys()

dict_keys(['sentids', 'imgid', 'sentences', 'split', 'filename'])

In [44]:
pprint.pprint(json_data['images'][0])

{'filename': '2513260012_03d33305cf.jpg',
 'imgid': 0,
 'sentences': [{'imgid': 0,
                'raw': 'A black dog is running after a white dog in the snow .',
                'sentid': 0,
                'tokens': ['a',
                           'black',
                           'dog',
                           'is',
                           'running',
                           'after',
                           'a',
                           'white',
                           'dog',
                           'in',
                           'the',
                           'snow']},
               {'imgid': 0,
                'raw': 'Black dog chasing brown dog through snow',
                'sentid': 1,
                'tokens': ['black',
                           'dog',
                           'chasing',
                           'brown',
                           'dog',
                           'through',
                           'snow']},
               